In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
import requests
from datetime import datetime, timedelta

# Colab에서 실행 중인지 확인
import sys
is_colab = 'google.colab' in sys.modules
if is_colab:
    print("Running in Colab")
else:
    print("Not running in Colab")

# 그래프를 노트북 내에 표시
#%matplotlib inline

Not running in Colab


In [2]:
# 한국 시간 예제 (KST)
kst_time_s = "2024-11-10 9:00:00"

# 문자열을 datetime 객체로 변환
kst_datetime_s = datetime.strptime(kst_time_s, '%Y-%m-%d %H:%M:%S')

# 한국 시간에서 9시간을 빼서 UTC로 변환
utc_datetime_s = kst_datetime_s - timedelta(hours=9)

# UTC 기준으로 유닉스 시간 계산
unix_time_s = int(kst_datetime_s.timestamp())

faildate = []

In [3]:
# 데이터 조회
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

device_id = 'wando01'
url = f'https://rojy53nt54.execute-api.ap-northeast-2.amazonaws.com/Prod/devices/{device_id}/sensors/oxygen'

def collect_sensor_data(device_id,start_date, end_date):
    
    headers = {
        "Content-Type": "application/json",
    }

    # 폼 데이터를 전송
    response = requests.get(url, params={"device_id": device_id, "b_time": start_date, "a_time": end_date, "size": 100000}, headers=headers)
    
    # 응답 출력
    print(response.status_code)

    
    # 응답 상태 코드 확인 및 JSON 데이터 가져오기
    if response.status_code == 200:
        data = response.json()

        if isinstance(data, list):
            df = pd.DataFrame(data)
        elif isinstance(data, dict) and 'result' in data and isinstance(data['result'], list):
            df = pd.DataFrame(data['result'])
        else:
            print("Unexpected data structure in the response")
            print("Response content:", data)

        if 'measure_time' in df.columns:
            df['date_time'] = pd.to_datetime(df['measure_time'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')
            
            
        #df['oxygen_ppm'] = df['oxygen_ppm'].apply(lambda x: x['value'] if isinstance(x, dict) else x)

        return df

    else:
        faildate.append((start_date,end_date))
        print("Failed to fetch data:", response.status_code)  



In [4]:
total_data = pd.DataFrame()
current_date = kst_datetime_s



while current_date < kst_datetime_s + timedelta(days=5):
    start_date = int(current_date.timestamp() * 1000)
    end_date = int((current_date + timedelta(days=3)).timestamp() * 1000)
    print(f"Fetching data from {start_date} to {end_date}")
    
    # 데이터 가져오기
    data_chunk = collect_sensor_data(device_id, start_date, end_date)
    total_data = pd.concat([total_data, data_chunk], ignore_index=True)  # 데이터 누적
    
    # 날짜를 3일 뒤로 이동
    current_date += timedelta(days=1)



Fetching data from 1731196800000 to 1731456000000
200
Fetching data from 1731283200000 to 1731542400000
200
Fetching data from 1731369600000 to 1731628800000


KeyboardInterrupt: 

In [31]:
# Excel 파일로 저장
output_filename = "C:/Users/lgiht/Downloads/output_data2.xlsx"
total_data.to_excel(output_filename, index=False)
print(f"Data saved to {output_filename}")

Data saved to C:/Users/lgiht/Downloads/output_data2.xlsx


In [17]:
print(faildate)
start_date = faildate[0][0]
end_date = faildate[0][1]
print(f"Fetching data from {start_date} to {end_date}")

# 데이터 가져오기
data_chunk = collect_sensor_data(device_id, start_date, end_date)
total_data = pd.concat([total_data, data_chunk], ignore_index=True)  # 데이터 누적

[(1723507200000, 1723766400000), (1723507200000, 1723766400000), (1723507200000, 1723766400000)]
Fetching data from 1723507200000 to 1723766400000
500
Failed to fetch data: 500
